In [3]:
import pandas as pd
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from auxiliar import add_messages

In [2]:
load_dotenv()

True

In [4]:
class SyntheticUserMessage(BaseModel):

    message: str = Field(
        ...,
        title="Message",
        description="The user message to generate for the target task intention.",
    )


class ListSyntheticUserMessages(BaseModel):

    messages: list[SyntheticUserMessage] = Field(
        ...,
        title="Messages",
        description="The list of synthetic user messages to generate for the target task intention.",
    )

output_parser = PydanticOutputParser(pydantic_object=ListSyntheticUserMessages)

In [5]:
system_prompt = """
You are tasked with generating synthetic user messages for a book recommendation system called Shelfmate.

The user intentions are:
{user_intentions}

Your task is to create {k} distinct messages for the following target task intention:
{target_task_intention}

Specific information about the target task intention:
{target_task_intention_description}

Follow these guidelines:
1. Focus exclusively on the target task intention, ensuring the message is relevant.
2. Each message should be between 5 and 20 words.
3. Avoid including any details or references to other user intentions.
4. Ensure the messages sound natural and typical of user queries for the given intention.
5. Follow the provided format strictly to maintain consistency.

Message format:
{format_instructions}
"""

prompt = PromptTemplate(
    template=system_prompt,
    input_variables=["k", "user_intentions", "target_task_intention" "target_task_intention_description", "format_instructions"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

In [6]:
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

user_intentions = ["update_profile_info", "insert_new_favorite_author_genre", "add_book_to_read_list", "suggest_authors_given_input", "suggest_authors_given_favorites", 
                   "suggest_books_given_input", "suggest_books_given_favorites", "suggest_books_given_trope", "browse_available_genres_books_authors", 
                   "create_reading_plan", "recommend_bookstores_per_district", "ask_about_chatbot_features", "ask_about_company_info"]

k = 50 

file_name = "synthetic_intetions.json"

synthetic_data_chain = prompt | llm | output_parser

# Intention 1 - Update User Profile Information

In [7]:
intention = "update_profile_info"

description = "The user wants to update a specific information about themselves in the platform, to do so they need to provide the information they want to update and the new value to replace the old info. \
               They can ask for updating the following informations: username, password, email and the district they live in. Note that the district must be in Portugal following the portuguese writting."

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

update_profile_info_messages = []
for message in response.messages:
    update_profile_info_messages.append({"Intention":intention, "Message":message.message})

Now you can check and edit your synthetic messages in a json file.

In [9]:
update_profile_info_messages

[{'Intention': 'update_profile_info',
  'Message': 'I want to update my username to BookLover123.'},
 {'Intention': 'update_profile_info',
  'Message': 'Please change my email to newemail@example.com.'},
 {'Intention': 'update_profile_info',
  'Message': 'Can you update my password for security reasons?'},
 {'Intention': 'update_profile_info',
  'Message': 'I need to change my district to Lisboa.'},
 {'Intention': 'update_profile_info',
  'Message': 'Update my username to ReadingQueen.'},
 {'Intention': 'update_profile_info',
  'Message': 'Change my email address to reader@mail.com.'},
 {'Intention': 'update_profile_info',
  'Message': 'I would like to update my password, please.'},
 {'Intention': 'update_profile_info',
  'Message': 'Please change my district to Porto.'},
 {'Intention': 'update_profile_info',
  'Message': 'Can you update my username to NovelFan?'},
 {'Intention': 'update_profile_info',
  'Message': 'I want to change my email to contact@myemail.com.'},
 {'Intention': 'u

In [10]:
add_messages(update_profile_info_messages, file_name)

# Intention 2 - Insert New Favorite Author or Genre

In [11]:
genres = pd.read_csv("C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/genres.csv").sample(10, random_state=42)["genre"].tolist()
authors = pd.read_csv("C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/authors.csv").sample(20, random_state=42)["author"].tolist()

In [12]:
intention = "insert_new_favorite_author_genre"

description = """The user intends to set an author or books genre as part of their favorite list. Users can have more than one book or genre in their favorite lists. \
                 The user should express clearly if they want to add an author or a genre and the name of it. \
                 A selection of the available genres are {genres}. \
                 A selection of the available authors are {authors}.
                """
description = description.format(genres=", ".join(genres), authors=", ".join(authors))
response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

insert_new_favorite_author_genre_messages = []

for message in response.messages:
    insert_new_favorite_author_genre_messages.append({"Intention":intention, "Message":message.message})

In [15]:
insert_new_favorite_author_genre_messages

[{'Intention': 'insert_new_favorite_author_genre',
  'Message': 'I want to add Leo Tolstoy as my favorite author.'},
 {'Intention': 'insert_new_favorite_author_genre',
  'Message': 'Please add the genre Technology & Engineering to my favorites.'},
 {'Intention': 'insert_new_favorite_author_genre',
  'Message': "I'd like to make Psychology one of my favorite genres."},
 {'Intention': 'insert_new_favorite_author_genre',
  'Message': 'Add Julie Garwood to my list of favorite authors.'},
 {'Intention': 'insert_new_favorite_author_genre',
  'Message': 'Can you include the genre History in my favorites?'},
 {'Intention': 'insert_new_favorite_author_genre',
  'Message': 'I want to add Paul J. Deitel as a favorite author.'},
 {'Intention': 'insert_new_favorite_author_genre',
  'Message': 'Please add the genre Performing Arts to my favorites.'},
 {'Intention': 'insert_new_favorite_author_genre',
  'Message': "I'd like to add Kim Lindros as my favorite author."},
 {'Intention': 'insert_new_favor

In [16]:
add_messages(insert_new_favorite_author_genre_messages, file_name)

# Intention 3 - Add a Book to my Read List

In [17]:
books = pd.read_csv("C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/books.csv").sample(20, random_state=42)["Title"].tolist()

In [30]:
intention = "add_book_to_read_list"


description = """The user intends to add a book they read and finished or started reading and stopped to their reading list. \
                The user must provide the title of the book, optionally the author, mandatorily the rating from 0 to 5 they want to give to the book \
                and if they did not finish the book they must say so. \
                This is a sample of the available books: {books}.
                """

description = description.format(books=", ".join(books))
response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

add_book_to_read_list_messages = []

for message in response.messages:
    add_book_to_read_list_messages.append({"Intention":intention, "Message":message.message})

In [32]:
add_book_to_read_list_messages

[{'Intention': 'add_book_to_read_list',
  'Message': "Add 'Aiding and Abetting: A Novel' to my reading list with a rating of 4."},
 {'Intention': 'add_book_to_read_list',
  'Message': "I want to add 'The Saboteurs' to my list and give it a 5."},
 {'Intention': 'add_book_to_read_list',
  'Message': "Please add 'Ohio Native Americans' to my reading list, rating it 3."},
 {'Intention': 'add_book_to_read_list',
  'Message': "Add 'Anne of the Island' to my read list, I rate it 5."},
 {'Intention': 'add_book_to_read_list',
  'Message': "I finished 'Desert Solitaire' and I want to rate it 4."},
 {'Intention': 'add_book_to_read_list',
  'Message': "Add 'The Giant, O'Brien' to my reading list, I rate it 2."},
 {'Intention': 'add_book_to_read_list',
  'Message': "Please add 'Making Sense Out of Suffering' to my list with a 3."},
 {'Intention': 'add_book_to_read_list',
  'Message': "I want to add 'A Home for Melanie' to my reading list, rating it 4."},
 {'Intention': 'add_book_to_read_list',
  'M

In [33]:
add_messages(add_book_to_read_list_messages, file_name)

# Intention 4 - Suggest new authors given favorites

In [46]:
intention = "suggest_authors_given_favorites"


description = """The user will ask for an author recommendation based on either their favorite books, genres or authors. \
                The user must clearly state that they want an author recommendation and state what will be the comparison object. \
                The user should not provide any specific name of author, genre or book.
                """

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

suggest_authors_given_favorites_messages = []

for message in response.messages:
    suggest_authors_given_favorites_messages.append({"Intention":intention, "Message":message.message})

In [49]:
suggest_authors_given_favorites_messages

[{'Intention': 'suggest_authors_given_favorites',
  'Message': 'Can you suggest authors based on my favorite books?'},
 {'Intention': 'suggest_authors_given_favorites',
  'Message': 'I need author recommendations that match my favorite genres.'},
 {'Intention': 'suggest_authors_given_favorites',
  'Message': 'Please recommend authors similar to my favorite authors.'},
 {'Intention': 'suggest_authors_given_favorites',
  'Message': 'What authors do you suggest if I like certain books?'},
 {'Intention': 'suggest_authors_given_favorites',
  'Message': 'Suggest some authors based on my preferred genres.'},
 {'Intention': 'suggest_authors_given_favorites',
  'Message': 'Can you find authors that align with my favorite reads?'},
 {'Intention': 'suggest_authors_given_favorites',
  'Message': 'I’m looking for authors similar to my favorites.'},
 {'Intention': 'suggest_authors_given_favorites',
  'Message': 'What authors would you recommend based on my book choices?'},
 {'Intention': 'suggest_au

In [50]:
add_messages(suggest_authors_given_favorites_messages, file_name)

# Intention 5 - Suggest new authors given user input

In [ ]:
intention = "suggest_authors_given_input"


description = """The user will ask for an author recommendation based on a specific book genre or genres, another author or authors, or on specific books. \
                The user must clearly state that they want an author recommendation and state what will be the comparison object. \
                This is a sample of the available books: {books}. \
                This is a sample of the available genres: {genres}. \
                This is a sample of the available authors: {authors}.
                """

description = description.format(books=", ".join(books), genres=', '.join(genres), authors=', '.join(authors))
response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

suggest_authors_given_input_messages = []

for message in response.messages:
    suggest_authors_given_input_messages.append({"Intention":intention, "Message":message.message})

In [42]:
suggest_authors_given_input_messages

[{'Intention': 'suggest_authors_given_input',
  'Message': 'Can you recommend authors similar to Leo Tolstoy?'},
 {'Intention': 'suggest_authors_given_input',
  'Message': "I loved 'Desert Solitaire'. Any author suggestions?"},
 {'Intention': 'suggest_authors_given_input',
  'Message': 'Suggest authors who write like Julie Garwood.'},
 {'Intention': 'suggest_authors_given_input',
  'Message': 'Who are some authors in the Psychology genre?'},
 {'Intention': 'suggest_authors_given_input',
  'Message': "I enjoyed 'The Giant, O'Brien'. Recommend authors."},
 {'Intention': 'suggest_authors_given_input',
  'Message': "Can you suggest authors based on 'Aiding and Abetting'?"},
 {'Intention': 'suggest_authors_given_input',
  'Message': 'Looking for authors similar to Sandra L. Hatch.'},
 {'Intention': 'suggest_authors_given_input',
  'Message': 'Recommend authors who write about Family & Relationships.'},
 {'Intention': 'suggest_authors_given_input',
  'Message': 'Who are some authors like Kei

In [43]:
add_messages(suggest_authors_given_input_messages, file_name)

# Intention 6 - Suggest new books given favorites

In [58]:
intention = "suggest_books_given_favorites"


description = """The user will ask for a book recommendation based on either their favorite books, genres or authors. \
                The user must clearly state that they want a book recommendation and state what will be the comparison object. \
                The user should not provide any specific name of author, genre or book.
                """

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

suggest_books_given_favorites_messages = []

for message in response.messages:
    suggest_books_given_favorites_messages.append({"Intention":intention, "Message":message.message})

In [59]:
suggest_books_given_favorites_messages

[{'Intention': 'suggest_books_given_favorites',
  'Message': 'Can you recommend books based on my favorite authors?'},
 {'Intention': 'suggest_books_given_favorites',
  'Message': 'I need book suggestions that align with my favorite genres.'},
 {'Intention': 'suggest_books_given_favorites',
  'Message': 'Please suggest books similar to my favorite reads.'},
 {'Intention': 'suggest_books_given_favorites',
  'Message': 'What books do you recommend based on my favorite books?'},
 {'Intention': 'suggest_books_given_favorites',
  'Message': "I'd like book recommendations that match my favorite authors."},
 {'Intention': 'suggest_books_given_favorites',
  'Message': 'Can you suggest books that fit my favorite genres?'},
 {'Intention': 'suggest_books_given_favorites',
  'Message': 'Recommend some books based on my favorite reads.'},
 {'Intention': 'suggest_books_given_favorites',
  'Message': 'What books would you suggest that are like my favorites?'},
 {'Intention': 'suggest_books_given_favo

In [60]:
add_messages(suggest_books_given_favorites_messages, file_name)

# Intention 7 - Suggest new books given user input

In [53]:
intention = "suggest_books_given_input"


description = """The user will ask for a book recommendation based on a specific genre or genres, an author or authors, or on other specific books. \
                The user must clearly state that they want an author recommendation and state what will be the comparison object. \
                This is a sample of the available books: {books}. \
                This is a sample of the available genres: {genres}. \
                This is a sample of the available authors: {authors}.
                """

description = description.format(books=", ".join(books), genres=', '.join(genres), authors=', '.join(authors))
response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

suggest_books_given_input_messages = []

for message in response.messages:
    suggest_books_given_input_messages.append({"Intention":intention, "Message":message.message})

In [54]:
suggest_books_given_input_messages

[{'Intention': 'suggest_books_given_input',
  'Message': "Can you suggest books similar to 'Desert Solitaire'?"},
 {'Intention': 'suggest_books_given_input',
  'Message': "I loved 'Anne of the Island'; what else should I read?"},
 {'Intention': 'suggest_books_given_input',
  'Message': "Recommend books like 'The Giant, O'Brien'."},
 {'Intention': 'suggest_books_given_input',
  'Message': "What books are similar to 'Making Sense Out of Suffering'?"},
 {'Intention': 'suggest_books_given_input',
  'Message': "Suggest books that are like 'Aiding and Abetting'."},
 {'Intention': 'suggest_books_given_input',
  'Message': "I enjoyed 'The Complete Chessplayer'; any recommendations?"},
 {'Intention': 'suggest_books_given_input',
  'Message': "What books would you suggest for fans of 'Ohio Native Americans'?"},
 {'Intention': 'suggest_books_given_input',
  'Message': "Can you recommend books like 'Spanish for Public Safety Personnel'?"},
 {'Intention': 'suggest_books_given_input',
  'Message': "

In [57]:
add_messages(suggest_books_given_input_messages, file_name)

# Intention 8 - Suggest new books given a trope

In [64]:
intention = "suggest_books_given_trope"


description = """The user will ask for a book recommendation based on a specific book trope or general book description. \
                The user must clearly state that they want an author recommendation and state what will be the comparison object. \
                """

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

suggest_books_given_trope_messages = []

for message in response.messages:
    suggest_books_given_trope_messages.append({"Intention":intention, "Message":message.message})

In [65]:
suggest_books_given_trope_messages

[{'Intention': 'suggest_books_given_trope',
  'Message': 'Can you suggest books with a friends-to-lovers trope?'},
 {'Intention': 'suggest_books_given_trope',
  'Message': 'I love enemies-to-lovers stories. Any recommendations?'},
 {'Intention': 'suggest_books_given_trope',
  'Message': 'What books feature a strong found family theme?'},
 {'Intention': 'suggest_books_given_trope',
  'Message': 'Please recommend books with a time travel trope.'},
 {'Intention': 'suggest_books_given_trope',
  'Message': 'I enjoy stories with a reluctant hero. Suggestions?'},
 {'Intention': 'suggest_books_given_trope',
  'Message': 'Can you suggest books that have a love triangle?'},
 {'Intention': 'suggest_books_given_trope',
  'Message': 'What are some good books with a second chance romance?'},
 {'Intention': 'suggest_books_given_trope',
  'Message': 'I like books with a magical realism element. Any ideas?'},
 {'Intention': 'suggest_books_given_trope',
  'Message': 'Please recommend books that involve 

In [66]:
add_messages(suggest_books_given_trope_messages, file_name)

# Intention 9 - Browse available genres, books, authors

In [67]:
intention = "browse_available_genres_books_authors"


description = """The user will ask to know which books, genres or authors are available in the platform. \
                The user must clearly state what they want to browse and can add filters such as 'Fiction books'. \
                """

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

browse_availables_genres_books_authors_messages = []

for message in response.messages:
    browse_availables_genres_books_authors_messages.append({"Intention":intention, "Message":message.message})

In [68]:
browse_availables_genres_books_authors_messages

[{'Intention': 'browse_available_genres_books_authors',
  'Message': 'What genres of books do you have available?'},
 {'Intention': 'browse_available_genres_books_authors',
  'Message': 'Can you show me the available fiction books?'},
 {'Intention': 'browse_available_genres_books_authors',
  'Message': "I'm looking for mystery authors on your platform."},
 {'Intention': 'browse_available_genres_books_authors',
  'Message': 'What are the top romance novels you offer?'},
 {'Intention': 'browse_available_genres_books_authors',
  'Message': 'List all available science fiction books.'},
 {'Intention': 'browse_available_genres_books_authors',
  'Message': 'Do you have any historical fiction authors?'},
 {'Intention': 'browse_available_genres_books_authors',
  'Message': 'Show me the available non-fiction genres.'},
 {'Intention': 'browse_available_genres_books_authors',
  'Message': 'What fantasy books can I browse?'},
 {'Intention': 'browse_available_genres_books_authors',
  'Message': 'Can

In [69]:
add_messages(browse_availables_genres_books_authors_messages, file_name)

# Intention 10 - Create Reading Plan

In [74]:
intention = "create_reading_plan"


description = """The user will ask to create a reading plan monthly or annual. \
                They can ask that plan to be based on their favorite authors, genres or books or based on specific authors, genres or books. \
                They can say if they are fast or slow readers and can state more than one genre, author or book. \
                They must clearly state if they want a monthly or annual plan and if they want it based on their favorites or specific authors, genres or books. \
                For the specific authors, genres and books, following are the samples of available ones: {authors}, {genres}, {books}.
                """

description = description.format(books=", ".join(books), genres=', '.join(genres), authors=', '.join(authors))
response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

create_reading_plan_messages = []

for message in response.messages:
    create_reading_plan_messages.append({"Intention":intention, "Message":message.message})

In [75]:
create_reading_plan_messages

[{'Intention': 'create_reading_plan',
  'Message': 'Can you create a monthly reading plan based on my favorite authors?'},
 {'Intention': 'create_reading_plan',
  'Message': 'I need an annual reading plan focused on historical fiction.'},
 {'Intention': 'create_reading_plan',
  'Message': 'Please make a reading plan for me with my favorite genres.'},
 {'Intention': 'create_reading_plan',
  'Message': "I'd like a monthly reading plan with specific books in mind."},
 {'Intention': 'create_reading_plan',
  'Message': "Create an annual reading plan based on Leo Tolstoy's works."},
 {'Intention': 'create_reading_plan',
  'Message': 'Can you suggest a reading plan for Psychology books this year?'},
 {'Intention': 'create_reading_plan',
  'Message': "I want a monthly plan featuring Sandra L. Hatch's novels."},
 {'Intention': 'create_reading_plan',
  'Message': 'Please create a reading plan for the next year with my favorites.'},
 {'Intention': 'create_reading_plan',
  'Message': "I'd like a r

In [76]:
add_messages(create_reading_plan_messages, file_name)

# Intention 11 - Recommend Bookstores

In [94]:
intention = "recommend_bookstores_per_district"


description = """The user will ask you to recommend bookstores in the district they live in or in other specific district in Portugal. \
                If the user asks for the district they live in, all the input must be in english. \
                If the user specifies a portuguese district, the district name must be written in portuguese spelling and the rest of the input in english.                 
                """

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

recommend_bookstores_messages = []

for message in response.messages:
    recommend_bookstores_messages.append({"Intention":intention, "Message":message.message})

In [95]:
recommend_bookstores_messages

[{'Intention': 'recommend_bookstores_per_district',
  'Message': 'Can you recommend bookstores in my district?'},
 {'Intention': 'recommend_bookstores_per_district',
  'Message': 'What are the best bookstores in Lisbon?'},
 {'Intention': 'recommend_bookstores_per_district',
  'Message': 'Please suggest some bookstores in Porto.'},
 {'Intention': 'recommend_bookstores_per_district',
  'Message': 'I need bookstore recommendations for my area.'},
 {'Intention': 'recommend_bookstores_per_district',
  'Message': 'Are there any good bookstores in Braga?'},
 {'Intention': 'recommend_bookstores_per_district',
  'Message': 'Can you find bookstores in Faro for me?'},
 {'Intention': 'recommend_bookstores_per_district',
  'Message': 'What bookstores do you recommend in Coimbra?'},
 {'Intention': 'recommend_bookstores_per_district',
  'Message': 'Suggest some bookstores in Aveiro.'},
 {'Intention': 'recommend_bookstores_per_district',
  'Message': 'Where can I find bookstores in Setúbal?'},
 {'Inte

In [96]:
add_messages(recommend_bookstores_messages, file_name)

# Intention 12 - Ask about chatbot features

In [98]:
intention = "ask_about_chatbot_features"


description = """The user wants to know more about Shelfmate and its chatbot features.
                It can include questions about the chatbot's capabilities, the platform's features, and the chatbot's limitations.
                """

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

chatbot_features_messages = []

for message in response.messages:
    chatbot_features_messages.append({"Intention":intention, "Message":message.message})

In [99]:
chatbot_features_messages

[{'Intention': 'ask_about_chatbot_features',
  'Message': 'What features does the Shelfmate chatbot offer?'},
 {'Intention': 'ask_about_chatbot_features',
  'Message': 'Can you tell me how the chatbot works?'},
 {'Intention': 'ask_about_chatbot_features',
  'Message': 'What can I ask the Shelfmate chatbot?'},
 {'Intention': 'ask_about_chatbot_features',
  'Message': "Are there any limitations to the chatbot's capabilities?"},
 {'Intention': 'ask_about_chatbot_features',
  'Message': 'How does the chatbot suggest books?'},
 {'Intention': 'ask_about_chatbot_features',
  'Message': 'Can the chatbot help me find new authors?'},
 {'Intention': 'ask_about_chatbot_features',
  'Message': 'What kind of recommendations can I get from the chatbot?'},
 {'Intention': 'ask_about_chatbot_features',
  'Message': 'Is the chatbot able to track my reading progress?'},
 {'Intention': 'ask_about_chatbot_features',
  'Message': 'How does the chatbot personalize suggestions for me?'},
 {'Intention': 'ask_ab

In [100]:
add_messages(chatbot_features_messages, file_name)

# Intention 13 - Ask about company information

In [102]:
intention = "ask_about_company_info"


description = """The user wants to know more about Shelfmate company.
                It can include questions about Shelfmate's mission, history, values, vision, ...
                """

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

company_info_messages = []

for message in response.messages:
    company_info_messages.append({"Intention":intention, "Message":message.message})

In [103]:
company_info_messages

[{'Intention': 'ask_about_company_info',
  'Message': "What is Shelfmate's mission statement?"},
 {'Intention': 'ask_about_company_info',
  'Message': 'Can you tell me about the history of Shelfmate?'},
 {'Intention': 'ask_about_company_info',
  'Message': 'What values does Shelfmate prioritize?'},
 {'Intention': 'ask_about_company_info',
  'Message': 'How did Shelfmate get started?'},
 {'Intention': 'ask_about_company_info',
  'Message': 'What is the vision of Shelfmate for the future?'},
 {'Intention': 'ask_about_company_info',
  'Message': 'Who founded Shelfmate and when?'},
 {'Intention': 'ask_about_company_info',
  'Message': 'What makes Shelfmate different from other book services?'},
 {'Intention': 'ask_about_company_info',
  'Message': "Can you explain Shelfmate's core values?"},
 {'Intention': 'ask_about_company_info',
  'Message': 'What is the purpose of Shelfmate?'},
 {'Intention': 'ask_about_company_info',
  'Message': 'How does Shelfmate support the reading community?'},
 

In [104]:
add_messages(company_info_messages, file_name)

# No Intention: None

In [110]:
system_prompt = """
You are tasked with generating synthetic user messages.

The user intentions are:
{user_intentions}

Your task is to create {k} distinct messages completely unrelated to the available user intentions.
These messages should be generic and not related to any specific task or intention.
The user is engaging in casual conversation.
The user might ask general questions, share opinions, or express emotions. 
The user might also ask for totaly none related questions to the platform. 
The user might ask general questions, share opinions, or express emotions.

Follow these guidelines:
1. Focus exclusively on not being related to any of the user intentions.
2. Each message should be between 5 and 20 words.
3. Avoid including any details or references to other user intentions.
4. Ensure the messages sound natural and typical of user queries for the given intention.
5. Follow the provided format strictly to maintain consistency.

Message format:
{format_instructions}
"""

In [111]:
prompt = PromptTemplate(
    template=system_prompt,
    input_variables=["k", "user_intentions"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

synthetic_data_chain = prompt | llm | output_parser

In [112]:
response = synthetic_data_chain.invoke({"k": (50), "user_intentions": user_intentions})

none_related_messages = []

for message in response.messages:
    none_related_messages.append({"Intention":"None", "Message":message.message})

In [113]:
none_related_messages

[{'Intention': 'None',
  'Message': "What's your favorite way to spend a rainy day?"},
 {'Intention': 'None',
  'Message': 'I just tried a new recipe, and it turned out great!'},
 {'Intention': 'None', 'Message': 'Do you think pineapple belongs on pizza?'},
 {'Intention': 'None',
  'Message': "I love watching the sunset; it's so calming."},
 {'Intention': 'None',
  'Message': 'Have you ever traveled to a place that changed your perspective?'},
 {'Intention': 'None',
  'Message': "What's the most interesting fact you've learned recently?"},
 {'Intention': 'None',
  'Message': "I can't believe how fast this year is going by!"},
 {'Intention': 'None',
  'Message': 'Do you prefer coffee or tea in the morning?'},
 {'Intention': 'None', 'Message': "What's your go-to comfort food?"},
 {'Intention': 'None',
  'Message': "I recently started a new hobby, and it's so much fun!"},
 {'Intention': 'None',
  'Message': 'What was the last movie that made you cry?'},
 {'Intention': 'None',
  'Message':

In [114]:
add_messages(none_related_messages, file_name)